<hr/>

# Data Mining
**Tamás Budavári** - budavari@jhu.edu <br/>

- Classification problems
- Naive Bayes Classifier
- Quadratic Discriminant Analysis

<hr/>

In [ ]:
%pylab inline

<h1><font color="darkblue">Classification</font></h1>
<hr/>

- Based on a **training set** of labeled points, assign class labels to unknown vectors in the **query set**.  

> **Training set**

>$T = \big\{ (\boldsymbol{x}_i, C_i) \big\}_{i=1}^N$ where $x_i\in \mathbb{R}^d$ and $C_i$ is the known class membership 

> **Query set**

>$Q = \big\{ \boldsymbol{x}_j \big\}_{j=1}^M$ where $x_j\in \mathbb{R}^d$ 

> For example,
> blood test results ($\boldsymbol{x}$) and sick/healty ($C$) - we want to predict if a new patient is sick based on the available measurements

- Similar to regression but with discrete categories to classify into...

### Classification Methods

- [$k$-NN](https://scikit-learn.org/stable/modules/neighbors.html)
- [Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html)
- [Quadratic Discriminant Analysis](https://scikit-learn.org/stable/modules/lda_qda.html)
- [Logistic regression](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)
- [Decisions trees](https://scikit-learn.org/stable/modules/tree.html)
- [Random forests](https://scikit-learn.org/stable/modules/ensemble.html#forests-of-randomized-trees)
- [Support Vector Machines](https://scikit-learn.org/stable/modules/svm.html)

### The Iris Dataset

We'll use this data set available in [scikit-learn](http://scikit-learn.org/stable/index.html), see [this](http://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html) page for details

### Other examples

More [exercises](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#examples-using-sklearn-neighbors-kneighborsclassifier) are available at http://scikit-learn.org

### $k$ Nearest Neighbors



In [ ]:
from sklearn import datasets

In [ ]:
# Load the data
iris = datasets.load_iris()

In [ ]:
# Unique known classes in training set
classes = np.unique(iris.target)
print ('There are %d classes:' % len(classes), classes)

In [ ]:
from sklearn import neighbors

In [ ]:
from matplotlib.colors import ListedColormap

In [ ]:
# Create color maps
cmap_light = ListedColormap(['#FFBBBB', '#BBFFBB', '#BBBBFF'])
cmap_bold = ListedColormap(['#CC0000', '#00AA00', '#0000CC'])

# Number of neighbors
n_neighbors = 15

# Import some data to play with
iris = datasets.load_iris()
X = iris.data[:,:2]  # we only take the first two features
y = iris.target

# step size in the mesh
h = 0.05

x_min, x_max = X[:,0].min()-1, X[:,0].max()+1
y_min, y_max = X[:,1].min()-1, X[:,1].max()+1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
grid = np.c_[xx.ravel(), yy.ravel()]

for weights in ['uniform', 'distance']:
    
    # we create an instance of Neighbours Classifier and fit the data.
    clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
    clf.fit(X, y)
    
    ZZ = clf.predict(grid)
    ZZ = ZZ.reshape(xx.shape) # 2-D grid layout

    # Plot the decision boundary. For that, we will assign a color to each grid point
    plt.figure(figsize=(5,5))
    plt.pcolormesh(xx, yy, ZZ, cmap=cmap_light)

    # Plot also the training points
    plt.scatter(X[:,0], X[:,1], c=y, cmap=cmap_bold)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title("3-Class classification (k = %i, weights = '%s')"
              % (n_neighbors, weights))

### Meaningful Distance?

- Need a [distance function](https://en.wikipedia.org/wiki/Distance#Mathematics)

> E.g., use Euclidean distance in $\mathbb{R}^d$

- Some examples

> $ \displaystyle \lVert X - Y \rVert_1 = \sum_{i=1}^{n} \lvert x_i - y_i \rvert $
>
> $ \displaystyle \lVert X - Y \rVert_2 = \left( \sum_{i=1}^{n} \lvert x_i - y_i \rvert^2 \right)^{1/2} $
>
> $ \displaystyle \lVert X - Y \rVert_p = \left( \sum_{i=1}^{n} \lvert x_i - y_i \rvert^p \right)^{1/p} \qquad 1 \le p < \infty $
>
> $ \displaystyle \lVert X - Y \rVert_\infty = \max_{i=1, \cdots, n} \lvert x_i - y_i \rvert $

- Problem with different features and units

> In practice, **centering** and **scaling** often helps <br/>
> Arguably, black art...


### Naive Bayes Classifier

- In general, we can use [Bayes' rule](https://en.wikipedia.org/wiki/Bayes%27_theorem#Statement_of_theorem) (and law of total probability) to infer discrete classes $C_k$ for a given $\boldsymbol{x}$ set of features

>$\displaystyle P(C_k \lvert\,\boldsymbol{x}) = \frac{\pi(C_k)\,{\cal{}L}_{\boldsymbol{x}}(C_k)}{Z} $ 


- Naively assuming the features are independent 

>$\displaystyle {\cal{}L}_{\boldsymbol{x}}(C_k) = \prod_{\alpha}^d p(x_{\alpha} \lvert C_k)$ 


### Naive Bayes: Learning

- Say for Gaussian likelihoods, we simply estimate the sample mean and variance of all features for each class $k$

>$\displaystyle p(x_{\alpha} \lvert C_k) = G(x_{\alpha};\mu_{k,\alpha}, \sigma^2_{k,\alpha})$

- We have to also pick some prior for the classes

> Using uniform or based on frequency of points in the training set?



### Naive Bayes: Estimation

- Look for maximum of the posterior


>$\displaystyle \hat{k} =  \mathrm{arg}\max_k \left[ \pi_k \prod_{\alpha}^d G(x_{\alpha};\mu_{k,\alpha}, \sigma^2_{k,\alpha})\right]$ 


In [ ]:
# Calculate feature means and variances for each class
param = dict()  # we save them in this dictionary
for k in classes:
    members = (iris.target == k) # boolean array
    num = members.sum()    # True:1, False:0
    prior = num / float(iris.target.size)
    X = iris.data[members,:] # slice out members
    mu = X.mean(axis=0)      # calc mean
    X -= mu
    var = (X*X).sum(axis=0) / (X[:,0].size-1)
    param[k] = (num, prior, mu, var) # save results
    print (k, mu, var)

In [ ]:
# init predicted values
k_pred = -1 * ones(iris.target.size)

# evaluate posterior for each point and find maximum
for i in range(iris.target.size):
    pmax, kmax = -1, None   # initialize to nonsense values
    for k in classes:
        num, prior, mu, var = param[k]
        diff = iris.data[i,:] - mu
        d2 = diff*diff / (2*var) 
        p = prior * np.exp(-d2.sum()) / np.sqrt(np.prod(2*pi*var))
        if p > pmax:
            pmax = p
            kmax = k
    k_pred[i] = kmax

print("Number of mislabeled points out of a total %d points : %d"
      % (iris.target.size, (iris.target!=k_pred).sum()))

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
# run sklearn's version - read up on differences if interested
gnb = GaussianNB()
y_pred = gnb.fit(iris.data, iris.target).predict(iris.data)

print("Number of mislabeled points out of a total %d points : %d"
      % (iris.target.size, (iris.target!=y_pred).sum()))

In [ ]:
# same results from both methods?
np.alltrue(k_pred==y_pred)

In [ ]:
# class probabilities
gnb.predict_proba(iris.data)

#### Still cheating!!

### Pros and Cons

- Features are automatically treated correctly relative to each other

> For example, measuring similar things in different units? 1m vs 1mm

> The estimated mean and variance puts them on a meaningful scale

- Independence is a strong assumption and for no good reason

> Actually... it helps with the computational cost!

### Exercise: Gaussian Naive Bayes

- Use the provided [training](Class-Train.csv) and [query](Class-Query.csv) sets to perform classification

> **Training** set consists of 3 columns of ($x_i$, $y_i$, $C_i$)

> **Query** set only has 2 columns of ($x_i$, $y_i$)


### Full Covariance Matrix

- Estimate the full covariance matrix for the classes

>$\displaystyle {\cal{}L}_{\boldsymbol{x}}(C_k) =  G(\boldsymbol{x};\mu_k, \Sigma_k)$

> Handles correlated features well

- Consider binary problem with 2 classes

> Taking the negative logarithm of the likelihoods we compare

>$\displaystyle (\boldsymbol{x}\!-\!\boldsymbol{\mu}_1)^T\,\Sigma_1^{-1}(\boldsymbol{x}\!-\!\boldsymbol{\mu}_1) + \ln\,\lvert\Sigma_1\lvert$ vs.

>$\displaystyle (\boldsymbol{x}\!-\!\boldsymbol{\mu}_2)^T\,\Sigma_2^{-1}(\boldsymbol{x}\!-\!\boldsymbol{\mu}_2) + \ln\,\lvert\Sigma_2\lvert$

> If the difference is lower than a threshold, we classify it accordingly

- This is called [**Quadratic Discriminant Analysis**](https://scikit-learn.org/stable/modules/lda_qda.html)

### Same Covariance Matrix

- When $\Sigma_1=\Sigma_2=\Sigma$, the quadratic terms cancel from the difference
 
>$\displaystyle (x\!-\!\mu_1)^T\,\Sigma^{-1}(x\!-\!\mu_1) $ 
>$\displaystyle -\ (x\!-\!\mu_2)^T\,\Sigma^{-1}(x\!-\!\mu_2) $

- Hence this is called [**Linear Discriminant Analysis**](https://scikit-learn.org/stable/modules/lda_qda.html)

> Fewer parameters to estimate during the learning process

> Good, if we don't have enough data, for example...

> Think linear vs quadratic fitting and how you decide between those

### Exercise: QDA and LDA

- Use the provided [training](Class-Train.csv) and [query](Class-Query.csv) sets to perform classification

> **Training** set consists of 3 columns of ($x_i$, $y_i$, $C_i$)

> **Query** set only has 2 columns of ($x_i$, $y_i$)



### Unhomework

- Visualize the results in the 2D features space
- Make these simple codes run faster by getting rid of the `for` loops, etc.


<h1><font color="darkblue">Summary</font></h1>
<hr/>

In [14]:
from sklearn import datasets

In [15]:
iris = datasets.load_iris()
c = np.unique(iris.target)
c

array([0, 1, 2])

### Procedure of [Naive Bayes](http://scikit-learn.org/stable/modules/naive_bayes.html)

- Fit
> Estimate the parameters in each class

- Predict
> For each unlabeled data, calculate the posterior for each class
>
> Classify the data with class k having the largest posterior

- Assumption
> Features are independent

In [16]:
# Toy example for Gaussian Naive Bayes

class GNB(dict):
    
    def fit(self, X, C):
        for k in np.unique(C):
            # Observation in class k
            members = (C == k)
            # Number of obvervation in class k
            num = members.sum() 
            # Use frequency as prior
            prior = num / float(C.size)
            # Choose the observation in class k
            XX = X[members,:] 
            # Calculate mean for class k
            mu = XX.mean(axis=0)
            # Center
            XX -= mu
            # Calculate variance for class k
            var = (XX*XX).sum(axis=0) / (XX.shape[0]-1)
            # Save the result for class k
            self[k] = (prior, num, mu, var)
    
    def predict(self, Y):
        pred = -1 * ones(Y.shape[0])
        for i in range(pred.size):
            # Initialization
            pmax, kmax = -1, None   
            # Calculate the posterior for each class
            for k in self:
                prior, num, mu, var = self[k]
                diff = Y[i,:] - mu
                d2 = diff*diff / (2*var) 
                posterior = prior * np.exp(-d2.sum()) / np.sqrt(np.prod(2*pi*var))
                # Update the threshold and prediction with the largest posterior
                if posterior > pmax:
                    pmax = posterior
                    kmax = k
                pred[i] = kmax
        return pred

In [17]:
clf = GNB()
clf.fit(iris.data, iris.target)
pred = clf.predict(iris.data)

print('Classifier: GNB')
print('Number of mislabeled points out of a total %d points : %d' % (iris.target.size, (iris.target!=pred).sum()))
print('Accuracy: ', mean(iris.target==pred))

Classifier: GNB
Number of mislabeled points out of a total 150 points : 6
Accuracy:  0.96


In [18]:
from sklearn.naive_bayes import GaussianNB

In [19]:
# Specify the model
clf = GaussianNB(priors=None)

# Fit
clf.fit(iris.data, iris.target)

# Predict
pred = clf.predict(iris.data)

print('Classifier: GNB')
print('Number of mislabeled points out of a total %d points : %d' % (iris.target.size, (iris.target!=pred).sum()))
print('Accuracy: ', mean(iris.target==pred))

Classifier: GNB
Number of mislabeled points out of a total 150 points : 6
Accuracy:  0.96
